# Raspado Web - Caso de Uso

## Paso 1 - Identificar la página objetivo

Identificar una página que sea raspable, la cuál tenga elementos estáticos en su contenido, es decir, que con una simple llamada obtenga los datos HTML.

[https://interceramic.com/mx_278VWPA/productos/articulos-de-cocina/accesorios.html](https://interceramic.com/mx_278VWPA/productos/articulos-de-cocina/accesorios.html)

Podemos inspeccionar su contenido HTML con el comando `! curl <url>`

In [ ]:
! curl https://interceramic.com/mx_278VWPA/productos/articulos-de-cocina/accesorios.html

> RESULTADO

```
<li class="item product product-item"> <div class="product-item-info product-out" data-container="product-grid">
<a href="https://interceramic.com/mx_278VWPA/canasta-y-contracanasta-universal-cocina-cromo-pulido.html" class="product photo product-item-photo" tabindex="-1">
<div class="entrepids-productlabel-wrapper promotion top-left 18871"><img src="/media/wysiwyg/kohler_premier/Etiqueta_Kohler_Web2.webp" alt="" /></div>
<span class="product-image-container" style="width:240px;">
<span class="product-image-wrapper" style="padding-bottom: 118.75%;">
<img class="product-image-photo" src="https://interceramic.com/media/catalog/product/cache/1d6c0ec6c124f51d05bca0fb360d0332/4/5/456_001_1.jpg" width="240" height="285" alt="Coladera para tarja línea Universal en cromo pulido" /></span>
</span>
</a>
<div class="product details product-item-details">
<div class="rating_bloq"></div>
<strong class="product name product-item-name">
<h2> <a class="product-item-link" href="https://interceramic.com/mx_278VWPA/canasta-y-contracanasta-universal-cocina-cromo-pulido.html">
Coladera para tarja línea Universal en cromo pulido </a></h2>
</strong>
<span class="bloq-info-color">Color</span>
<div class="price-box price-final_price" data-role="priceBox" data-product-id="18871" data-price-box="product-id-18871">
<span class="price-container price-final_price tax weee">
<span id="product-price-18871" data-price-amount="540" data-price-type="finalPrice" class="price-wrapper ">
<span class="bloq-container-price">
<span class="bloq-left-price">
<span class="price">$540.00 MXN </span> </span>

</span>
</span>
<span>
<span> por pieza</span>
</span>
</span>
</div>

</div>
</div>

</li>
```

## Paso 2 - Identificar las etiquetas que contienen la información a recuperar

Identificaremos, primero la etiqueta padre que contiene toda la información agrupada, en este caso se trata de una etiqueta tipo `<li class="item product product-item">` que contiene toda información de un producto (imagen, título, precio, etc.) y es repetitivo en la lista de productos.

Y a partir de esa etiqueta identificamos todas las otras etiquetas anidadas con la información.

**Imagen del producto**

`<img class="product-image-photo" ...>`

**Enlace a la información del producto**

`<a class="product-item-link" ...>`

**Nombre del producto**

`<strong class="product name product-item-name">`

**Precio del producto**

`<span class="price">`

## Paso 3 - Planificar recuperar los elementos con `xpath`

> XPATH Doc

[https://www.w3schools.com/xml/xpath_intro.asp](https://www.w3schools.com/xml/xpath_intro.asp)

**Seleccionar un nodo desde la ruta**

* `//` - Selecciona un elemento desde cualquier lugar
* `/` - Selecciona un elemento desde la raíz
* `./` - Selecciona un elemento a partir del nodo actual
* `../` - Selecciona un elemento a partir del nodo padre
* `../../` - Selecciona un elemento a partir del nodo ancestro al padre
* `.//` - Selecciona un elemento a partir del nodo actual en cualquier anidación

**Seleccionar el nodo según sus características**

* `TAGNAME` - Selecciona un nodo de tipo `TAGNAME`
* `TAGNAME[@class='CLASE']` - Selecciona un nodo de tipo `TAGNAME` con la clase exactamente igual a `CLASE`
* `*[@class='CLASE']` - Selecciona un nodo de cualquier tipo de etiqueta con la clase exactamente igual a `CLASE`
* `TAGNAME[@id='ID']` - Selecciona un nodo de tipo `TAGNAME` con el atributo `id="ID"`
* `TAGNAME[starts-with(@ATRIBUTO, 'VALOR')]` - Selecciona un nodo de tipo `TAGNAME` que su atributo `ATRIBUTO`, comienza con el `VALOR`, por ejemplo, `div[starts-with(@class, 'item')]`
* `TAGNAME[contains(@ATRIBUTO, 'VALOR')]` - Selecciona un nodo de tipo `TAGNAME` que su atributo `ATRIBUTO`, contenga con el `VALOR`, por ejemplo, `div[contains(@class, 'product')]`

**Recuperar el texto o atributo de la etiqueta**

* `/text()` - Recupera el texto de la etiqueta
* `/@src` - Recupera el valor del atributo `src`, que es el enlace a la imagen
* `/@ATRIBUTO` - Recupera el atributo `ATRIBUTO` en general, por ejemplo, `@href` devuelve la url de un enlace

> Ejemplo

```py
for item in response.xpath("//li[@class='item product product-item']"):
  imagen = item.xpath(".//img[@class='product-image-photo']/@src").get()
  nombre = item.xpath(".//strong[@class='product name product-item-name']/text()")
  enlace = item.xpath(".//a[@class='product-item-link']/@href").get()
  precio = item.xpath(".//span[@class='price']/text()").get()
```

## Paso 4 - Configurar a `scrapydo`

> Instalar la librería `scrapydo`

In [ ]:
! pip install scrapydo

> Configurar la librería

In [3]:
import scrapydo

scrapydo.setup()

## Paso 5 - Crear la función de raspado

In [8]:
def parser(response):
  for item in response.xpath("//li[@class='item product product-item']"):
    imagen = item.xpath(".//img[@class='product-image-photo']/@src").get()
    nombre = item.xpath(".//a[@class='product-item-link']/text()").get()
    enlace = item.xpath(".//a[@class='product-item-link']/@href").get()
    precio = item.xpath(".//span[@class='price']/text()").get()

    yield {
        "nombre": nombre,
        "precio": precio,
        "enlace": enlace,
        "imagen": imagen
    }

## Paso 6 - Adquirimos los datos retenidos

Recuperamos los valores retenidos por `yield`.

In [9]:
productos = scrapydo.crawl("https://interceramic.com/mx_278VWPA/productos/articulos-de-cocina/accesorios.html", callback=parser)

productos

[{'nombre': '\nDispensador de jabón universal en negro mate ',
  'precio': '$2,990.00 MXN ',
  'enlace': 'https://interceramic.com/mx_278VWPA/dispensador-de-jabon-universal-en-negro-mate.html',
  'imagen': 'https://interceramic.com/media/catalog/product/cache/1d6c0ec6c124f51d05bca0fb360d0332/7/2/7200_001_2.jpg'},
 {'nombre': '\nColadera para tarja línea Universal en cromo pulido ',
  'precio': '$540.00 MXN ',
  'enlace': 'https://interceramic.com/mx_278VWPA/canasta-y-contracanasta-universal-cocina-cromo-pulido.html',
  'imagen': 'https://interceramic.com/media/catalog/product/cache/1d6c0ec6c124f51d05bca0fb360d0332/4/5/456_001_1.jpg'},
 {'nombre': '\nDispensador de jabón línea Universal en cromo pulido ',
  'precio': '$1,740.00 MXN ',
  'enlace': 'https://interceramic.com/mx_278VWPA/contemporary-dispensador-de-jabon-cromo-pulido.html',
  'imagen': 'https://interceramic.com/media/catalog/product/cache/1d6c0ec6c124f51d05bca0fb360d0332/4/6/465_001_1.jpg'},
 {'nombre': '\nColadera para tarj

## Paso 7 - Convertir los resultados en un conjunto de datos

Creamos un DataFrame de pandas a partir de la lista de productos extraídos.

In [11]:
import pandas as pd

data = pd.DataFrame(productos)

# Limpiamos la columna nombre, para remover espacios y saltos de línea
# al principio y al final con <texto>.strip()
data["nombre"] = data["nombre"].map(lambda nombre: nombre.strip())

data.sample(5)

,nombre,precio,enlace,imagen
11,Rejilla para tarja línea Whitehaven,"$4,305.00 MXN",https://interceramic.com/mx_278VWPA/rejilla-wh...,https://interceramic.com/media/catalog/product...
7,Escurridor de copas Kohler,$990.00 MXN,https://interceramic.com/mx_278VWPA/accesorio-...,https://interceramic.com/media/catalog/product...
6,Escurridor para tarja línea Universal en blanco,$840.00 MXN,https://interceramic.com/mx_278VWPA/rack-unive...,https://interceramic.com/media/catalog/product...
5,Cubierta para tarja de cocina línea Universal ...,$789.99 MXN,https://interceramic.com/mx_278VWPA/cubierta-u...,https://interceramic.com/media/catalog/product...
8,Porta servilletas para cocina línea Universal ...,$840.00 MXN,https://interceramic.com/mx_278VWPA/porta-pape...,https://interceramic.com/media/catalog/product...


## Paso 8 - Guardamos el conjuntos de datos en CSV

Exportamos a un archivo los datos recuperados.

In [12]:
data.to_csv("interceramic_cocina_accesorios_pagina_1.csv", index=False)